#### **Clone Vision Utilities for Faster R-CNN Training**

In [1]:
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp vision/references/detection/utils.py ./

!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 591764, done.
remote: Counting objects: 100% (2042/2042), done.
remote: Compressing objects: 100% (953/953), done.
remote: Total 591764 (delta 1792), reused 1148 (delta 1084), pack-reused 589722 (from 7)
Receiving objects: 100% (591764/591764), 1.12 GiB | 46.33 MiB/s, done.
Resolving deltas: 100% (552848/552848), done.
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


#### **Import Required Libraries**

In [2]:
import os
import shutil
import logging
import random
import warnings
import time
import io
import sys
from tqdm import tqdm
from pathlib import Path

# Data handling and transformations
import cv2
import numpy as np
from PIL import Image
import xml.etree.ElementTree as ET

# Machine learning and deep learning libraries
import torch
import torchvision
from torchvision import transforms as torchtrans
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

# For image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# Custom libraries
import utils
import transforms as T
import engine

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


#### **Organize and Split Dataset for Traning**

In [3]:
data_dir = "/kaggle/input/sh17-dataset-for-ppe-detection"
output_dir = "/kaggle/working/data"

train_txt = os.path.join(data_dir, "train_files.txt")
val_txt = os.path.join(data_dir, "val_files.txt")

os.makedirs(os.path.join(output_dir, "train", "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "train", "labels"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "val", "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "val", "labels"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "test", "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "test", "labels"), exist_ok=True)

# Read file paths
with open(train_txt, "r") as f:
    train_files = f.read().splitlines()

with open(val_txt, "r") as f:
    test_files = f.read().splitlines()

# Split train into train and val (80:20)
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42, shuffle=True)
print(f"Train: {len(train_files)} | Val: {len(val_files)} | Test: {len(test_files)}")

# Copy files to respective folders
def copy_files(file_list, src_images, src_labels, dst_images, dst_labels):
    for file in tqdm(file_list):
        image_file = os.path.join(src_images, file)
        label_file = os.path.join(src_labels, file.split(".")[0] + ".xml")

        # Copy images and labels
        if os.path.exists(image_file):
            shutil.copy(image_file, dst_images)
        else:
            print(f"Image not found: {image_file}")

        if os.path.exists(label_file):
            shutil.copy(label_file, dst_labels)
        else:
            print(f"Label not found: {label_file}")

# Define source and destination folders
src_images = os.path.join(data_dir, "images")
src_labels = os.path.join(data_dir, "voc_labels")

print("Copying train files...")
copy_files(train_files, src_images, src_labels,
           os.path.join(output_dir, "train", "images"),
           os.path.join(output_dir, "train", "labels"))

print("Copying val files...")
copy_files(val_files, src_images, src_labels,
           os.path.join(output_dir, "val", "images"),
           os.path.join(output_dir, "val", "labels"))

print("Copying test files...")
copy_files(test_files, src_images, src_labels,
           os.path.join(output_dir, "test", "images"),
           os.path.join(output_dir, "test", "labels"))

print("Data prepared!")

Train: 5183 | Val: 1296 | Test: 1620
Copying train files...


100%|██████████| 5183/5183 [02:49<00:00, 30.54it/s]


Copying val files...


100%|██████████| 1296/1296 [00:43<00:00, 30.11it/s]


Copying test files...


100%|██████████| 1620/1620 [00:54<00:00, 29.52it/s]

Data prepared!


#### **Validate and Update Invalid Bounding Boxes**

In [4]:
# ========================================
# Check and update invalid bounding boxes
# ========================================

class BBoxValidator:
    def __init__(self, data_dir):
        """
        Initialize the BBoxValidator with the directory containing XML files.

        Parameters:
            data_dir: Path to the base directory containing train, val, and test folders.
        """
        self.data_dir = data_dir

    def validate_bbox(self, file_path):
        """
        Validate bounding boxes in a given XML file.

        Parameters:
            file_path (str): Path to the XML file.

        Returns:
            list: A list of invalid bounding boxes, each represented as a tuple (xmin, xmax, ymin, ymax).
        """
        invalid_bboxes = []
        try:
            tree = ET.parse(file_path)
            root = tree.getroot()
            size = root.find('size')
            image_width = int(size.find('width').text)
            image_height = int(size.find('height').text)

            for obj in root.findall('object'):
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                xmax = int(bndbox.find('xmax').text)
                ymin = int(bndbox.find('ymin').text)
                ymax = int(bndbox.find('ymax').text)

                # Check for invalid bounding box conditions
                if xmin < 0 or ymin < 0 or xmax > image_width or ymax > image_height or xmin >= xmax or ymin >= ymax:
                    invalid_bboxes.append((xmin, xmax, ymin, ymax))
        except Exception as e:
            print(f"Error validating {file_path}: {e}")

        return invalid_bboxes

    def find_invalid_files(self, labels_dir):
        """
        Find all XML files with invalid bounding boxes in a given directory.

        Parameters:
            labels_dir (str): Path to the directory containing XML label files.

        Returns:
            list: A list of tuples (file_name, invalid_bboxes).
        """
        invalid_files = []
        for file_name in os.listdir(labels_dir):
            if file_name.endswith('.xml'):
                file_path = os.path.join(labels_dir, file_name)
                invalid_bboxes = self.validate_bbox(file_path)
                if invalid_bboxes:
                    invalid_files.append((file_name, invalid_bboxes))
        return invalid_files

    def update_invalid_bboxes(self, labels_dir):
        """
        Update invalid bounding boxes in all XML files within the given directory.

        Parameters:
            labels_dir (str): Path to the directory containing XML label files.
        """
        for file_name in os.listdir(labels_dir):
            if file_name.endswith('.xml'):
                file_path = os.path.join(labels_dir, file_name)
                try:
                    tree = ET.parse(file_path)
                    root = tree.getroot()
                    size = root.find('size')
                    image_width = int(size.find('width').text)
                    image_height = int(size.find('height').text)

                    updated = False

                    for obj in root.findall('object'):
                        bbox = obj.find('bndbox')
                        if bbox is None:
                            continue

                        xmin = int(bbox.find('xmin').text)
                        ymin = int(bbox.find('ymin').text)
                        xmax = int(bbox.find('xmax').text)
                        ymax = int(bbox.find('ymax').text)

                        # Correct invalid bounding boxes
                        if xmin < 0:
                            bbox.find('xmin').text = '0'
                            updated = True
                        if ymin < 0:
                            bbox.find('ymin').text = '0'
                            updated = True
                        if xmax > image_width:
                            bbox.find('xmax').text = str(image_width)
                            updated = True
                        if ymax > image_height:
                            bbox.find('ymax').text = str(image_height)
                            updated = True
                        if xmin >= xmax:
                            bbox.find('xmax').text = str(xmin + 1)
                            updated = True
                        if ymin >= ymax:
                            bbox.find('ymax').text = str(ymin + 1)
                            updated = True

                    if updated:
                        tree.write(file_path)
                        print(f"    => Updated invalid bounding boxes in {file_path}")

                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")

    def process_all_folders(self):
        """
        Validate and update bounding boxes in train, val, and test label folders.
        """
        for folder in ['train', 'val', 'test']:
            labels_dir = os.path.join(self.data_dir, folder, 'labels')
            print(f"\nProcessing {labels_dir}...")

            # Find and print invalid files
            invalid_files = self.find_invalid_files(labels_dir)
            if invalid_files:
                print(f"    Files with invalid bounding boxes in {folder}:")
                for file_name, bboxes in invalid_files:
                    print(f"        File: {file_name}")
                    for bbox in bboxes:
                        print(f"              Invalid BBox: {bbox}")
            else:
                print(f"    No invalid bounding boxes found in {folder}.")

            # Update invalid bounding boxes
            self.update_invalid_bboxes(labels_dir)

In [5]:
data_dir = '/kaggle/working/data'
validator = BBoxValidator(data_dir)
validator.process_all_folders()


Processing /kaggle/working/data/train/labels...
    Files with invalid bounding boxes in train:
        File: pexels-photo-258626.xml
              Invalid BBox: (1725, 1874, 4430, 4664)
        File: pexels-photo-259265.xml
              Invalid BBox: (1559, 3509, 123, 1358)
    => Updated invalid bounding boxes in /kaggle/working/data/train/labels/pexels-photo-258626.xml
    => Updated invalid bounding boxes in /kaggle/working/data/train/labels/pexels-photo-259265.xml

Processing /kaggle/working/data/val/labels...
    No invalid bounding boxes found in val.

Processing /kaggle/working/data/test/labels...
    No invalid bounding boxes found in test.


#### **Define Custom Dataset Class**

In [6]:
# ========================================
# Class Custom Dataset
# ========================================
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, class_mapping, transforms=None):
        self.root = root
        self.class_mapping = class_mapping
        self.transforms = transforms
        self.images = sorted(os.listdir(os.path.join(root, "images")))  # Sort images
        self.labels = sorted(os.listdir(os.path.join(root, "labels")))  # Sort labels

        # Ensure the same number of images and labels
        assert len(self.images) == len(self.labels), "Mismatch between number of images and labels"

        # Optionally ensure filenames (without extensions) match between images and labels
        for img, label in zip(self.images, self.labels):
            assert img.split('.')[0] == label.split('.')[0], f"Image {img} and label {label} do not match"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.images[idx])
        label_path = os.path.join(self.root, "labels", self.labels[idx])
        img = Image.open(img_path).convert("RGB")
        tree = ET.parse(label_path)
        root = tree.getroot()

        boxes = []
        labels = []
        img_width, img_height = img.size
        for obj in root.findall("object"):
            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)
            boxes.append([xmin, ymin, xmax, ymax])
            name = obj.find("name").text

            if name in self.class_mapping:
                labels.append(self.class_mapping[name])
            else:
                raise ValueError(f"Unknown label '{name}' in {label_path}")

        # Convert boxes to numpy array for albumentations (no manual normalization here)
        boxes = np.array(boxes, dtype=np.float32)

        # Apply transformations if provided
        if self.transforms:
            # albumentations requires input to be a dictionary with 'image' and 'bboxes'
            augmented = self.transforms(image=np.array(img), bboxes=boxes, labels=labels)
            img = augmented['image']
            boxes = augmented['bboxes']

        # Convert boxes to tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        # Calculate area of each box
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # Suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        # Prepare the target dictionary
        target = {
            "boxes": boxes,
            "labels": labels,
            "area": area,
            "iscrowd": iscrowd,
            "image_id": int(idx)
        }

        return img, target



# =================================
# Data Loaders
# =================================
def collate_fn(batch):
    return tuple(zip(*batch))

def get_dataloaders(data_dir, class_mapping, batch_size=4):
    # Define albumentations transformations
    image_transforms = A.Compose(
        [
            A.Resize(640, 640),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'])
    )



    # Create dataset instances
    train_dataset = CustomDataset(os.path.join(data_dir, "train"), class_mapping, transforms=image_transforms)
    val_dataset = CustomDataset(os.path.join(data_dir, "val"), class_mapping, transforms=image_transforms)
    test_dataset = CustomDataset(os.path.join(data_dir, "test"), class_mapping, transforms=image_transforms)

    # Create data loaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    return train_loader, val_loader, test_loader

#### **Define dataloader with Weighted Random Sampler** 

In [9]:
import pickle
from collections import Counter
from torch.utils.data import WeightedRandomSampler

def get_weighted_random_sampler(dataset, class_mapping):
    """
    Create a WeightedRandomSampler for weighted sampling based on class distribution.

    Args:
        dataset (torch.utils.data.Dataset): The dataset to sample from.
        class_mapping (dict): A dictionary mapping class names to class indices.

    Returns:
        WeightedRandomSampler: A sampler that samples based on the calculated weights.
    """
    # Collect all labels in the dataset
    all_labels = []
    for _, target in tqdm(dataset, desc="Collecting labels"):
        all_labels.extend(label.item() for label in target["labels"])

    # Count the occurrences of each class
    class_counts = Counter(all_labels)

    # Calculate class weights (inverse proportional to class frequency)
    total_samples = sum(class_counts.values())
    class_weights = {
        cls_id: (total_samples / count if count > 0 else 0)
        for cls_id, count in class_counts.items()
    }
    print("Class weights:", class_weights)

    # File to save/load class weights
    weights_file = "/kaggle/working/class_weights.pkl"

    # Load weights if file exists; otherwise, save newly computed weights
    if os.path.exists(weights_file):
        with open(weights_file, "rb") as f:
            class_weights = pickle.load(f)
        print(f"Loaded class weights from {weights_file}")
    else:
        with open(weights_file, "wb") as f:
            pickle.dump(class_weights, f)
        print(f"Saved class weights to {weights_file}")

    # Compute weights for each sample in the dataset
    weights = []
    for _, target in tqdm(dataset, desc="Computing sample weights"):
        if len(target["labels"]) > 0:
            # Average the class weights for all labels in a sample
            sample_weight = sum(class_weights[label.item()] for label in target["labels"]) / len(target["labels"])
        else:
            sample_weight = 0
        weights.append(sample_weight)

    # Ensure the weights match the dataset size
    print(f"Length of weights: {len(weights)}")
    print(f"Length of dataset: {len(dataset)}")
    assert len(weights) == len(dataset), "Weights and dataset size must match!"

    # Create a WeightedRandomSampler
    sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

    return sampler


def get_dataloaders_with_weighted_sampling(data_dir, class_mapping, batch_size):
    """
    Prepare DataLoaders with WeightedRandomSampler for training.

    Args:
        data_dir (str): Path to the dataset directory containing train, val, and test folders.
        class_mapping (dict): A dictionary mapping class names to class indices.
        batch_size (int): The number of samples per batch.

    Returns:
        Tuple[torch.utils.data.DataLoader, torch.utils.data.DataLoader, torch.utils.data.DataLoader]:
            DataLoaders for training, validation, and testing datasets.
    """
    # Define image transformations using Albumentations
    image_transforms = A.Compose(
        [
            A.Resize(640, 640),  # Resize images to 640x640
            A.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize with ImageNet mean/std
                        std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
            ToTensorV2(),  # Convert to PyTorch tensor
        ],
        bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'])  # Bounding box parameters
    )

    # Initialize datasets
    train_dataset = CustomDataset(os.path.join(data_dir, "train"), class_mapping, transforms=image_transforms)
    val_dataset = CustomDataset(os.path.join(data_dir, "val"), class_mapping, transforms=image_transforms)
    test_dataset = CustomDataset(os.path.join(data_dir, "test"), class_mapping, transforms=image_transforms)

    # Create WeightedRandomSampler for the training dataset
    print("Initializing WeightedRandomSampler for training dataset...")
    start_time = time.time()
    train_sampler = get_weighted_random_sampler(train_dataset, class_mapping)
    end_time = time.time()
    print(f"Sampler creation completed in {end_time - start_time:.2f} seconds")

    # Create DataLoaders
    print("Creating DataLoaders...")
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler, collate_fn=collate_fn
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
    )

    print("DataLoaders created successfully!")
    return train_loader, val_loader, test_loader

#### **Setup Logging, Class Mapping, and Model Initialization**

In [ ]:
from coco_utils import get_coco_api_from_dataset
from coco_eval import CocoEvaluator
from engine import train_one_epoch, evaluate

# ========================
# Logging Configuration
# ========================
# Truy cập root và Loại bỏ tất cả các handlers hiện có
root_logger = logging.getLogger()
for handler in root_logger.handlers[:]:
    root_logger.removeHandler(handler)

LOG_FILE = "/kaggle/working/log.txt"
Path(LOG_FILE).parent.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s",
                    handlers=[
                        logging.FileHandler(LOG_FILE, mode='w'),
                        logging.StreamHandler()  # Display on terminal
                    ])
logger = logging.getLogger()


# ========================
# Define Class Mapping
# ========================
class_mapping = {
    "__background__": 0,
    "person": 1,
    "ear": 2,
    "ear-mufs": 3,
    "face": 4,
    "face-guard": 5,
    "face-mask-medical": 6,
    "foot": 7,
    "tools": 8,
    "glasses": 9,
    "gloves": 10,
    "helmet": 11,
    "hands": 12,
    "head": 13,
    "medical-suit": 14,
    "shoes": 15,
    "safety-suit": 16,
    "safety-vest": 17
}


# ========================
# Get Model
# ========================
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="COCO_V1")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    print("Model Done!")
    return model

#### **Train and Evaluate Model**

In [10]:
def train_and_evaluate(data_dir: str, num_epochs: int = 10, save_dir: str = 'models', sampling=True):
    batch_size = 16
    num_classes = len(class_mapping)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")

    if sampling:
        train_loader, val_loader, test_loader = get_dataloaders_with_weighted_sampling(data_dir, class_mapping, batch_size)
    else:
        train_loader, val_loader, test_loader = get_dataloaders(data_dir, class_mapping, batch_size)

    # Initialize Model
    model = get_model(num_classes)
    model.to(device)

    # Optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

    # Training Loop
    for epoch in range(num_epochs):
        logger.info(f"Starting epoch {epoch + 1}/{num_epochs}")

        # Train One Epoch
        train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=20)

        # Evaluate on Validation Set
        logger.info("Evaluating on validation set...")
        metrics = evaluate(model, val_loader, device)
        logger.info(f"Validation metrics after epoch {epoch + 1}: {metrics}")

    # Final Evaluation on Test Set
    logger.info("Evaluating on test set...")
    final_metrics = evaluate(model, test_loader, device)
    logger.info(f"Final evaluation metrics: {final_metrics}")
    print(final_metrics)

    # Save Model
    model_save_path = os.path.join(save_dir, "fasterrcnn_model.pth")
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

    return model, final_metrics


In [11]:
if __name__ == '__main__':
    try:
        data_dir = "/kaggle/working/data"
        save_dir = "/kaggle/working/models"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        num_epochs = 10

        model, final_metrics = train_and_evaluate(data_dir, num_epochs, save_dir, sampling=True)

    except Exception as e:
        logger.error(f"Program failed: {str(e)}")
        raise

2025-01-14 06:48:27,536 - INFO - Using device: cuda


Initializing WeightedRandomSampler for training dataset...


Class weights: {13: 6.313592486303157, 4: 8.534649973549639, 2: 9.930242100943783, 12: 4.779302853757282, 10: 26.963788300835656, 17: 143.19526627218934, 15: 17.108518911276068, 11: 78.57142857142857, 16: 338.46153846153845, 1: 5.448609703928853, 8: 16.296296296296298, 9: 40.333333333333336, 6: 120.09925558312655, 3: 234.95145631067962, 5: 576.1904761904761, 7: 97.18875502008032, 14: 498.96907216494844}
Saved class weights to /kaggle/working/class_weights.pkl


Computing sample weights: 100%|██████████| 5183/5183 [29:00<00:00,  2.98it/s]


Length of weights: 5183
Length of dataset: 5183
Sampler creation completed in 3474.49 seconds
Creating DataLoaders...
DataLoaders created successfully!


...

Epoch: [9]  [323/324]  eta: 0:00:07  lr: 0.010000  loss: 0.3494 (0.3587)  loss_classifier: 0.1068 (0.1078)  loss_box_reg: 0.1919 (0.1887)  loss_objectness: 0.0080 (0.0097)  loss_rpn_box_reg: 0.0391 (0.0525)  time: 7.9495  data: 6.4954  max mem: 11520
Epoch: [9] Total time: 0:42:06 (7.7975 s / it)
creating index...
index created!
Test:  [ 0/81]  eta: 0:07:38  model_time: 0.7019 (0.7019)  evaluator_time: 0.0469 (0.0469)  time: 5.6630  data: 4.8954  max mem: 11520
Test:  [80/81]  eta: 0:00:07  model_time: 0.6904 (0.6920)  evaluator_time: 0.0440 (0.0491)  time: 7.3495  data: 6.5904  max mem: 11520
Test: Total time: 0:09:33 (7.0847 s / it)
Averaged stats: model_time: 0.6904 (0.6920)  evaluator_time: 0.0440 (0.0491)
Accumulating evaluation results...


2025-01-14 17:19:15,888 - INFO - Validation metrics after epoch 10: <coco_eval.CocoEvaluator object at 0x7be9024ae6b0>
2025-01-14 17:19:15,889 - INFO - Evaluating on test set...


DONE (t=1.16s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.271
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.494
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.267
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.117
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.281
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.333
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.241
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.364
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.366
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.172
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.373
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.428
creatin

2025-01-14 17:42:28,003 - INFO - Final evaluation metrics: <coco_eval.CocoEvaluator object at 0x7be827cf05b0>


DONE (t=1.15s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.271
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.492
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.265
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.103
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.308
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.235
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.371
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.373
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.182
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.400
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.420
Model s